In [1]:
# !pip install torch torchvision
# !pip install git+https://github.com/openai/CLIP.git


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   --- ------------------------------------ 0.1/1.2 MB 3.0 MB/s eta 0:00:01
   --------------- ------------------------ 0.5/1.2 MB 5.8 MB/s eta 0:00:01
   ------------------------------ --------- 0.9/1.2 MB 7.1 MB/s eta 0:00:01
   ---------------------------------------- 1.2/1.2 MB 8.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/159.7 MB ? eta -:--:--
   ---------------------------------------- 0.5/159.7 MB 10.9 MB/s eta 0:00:15
   ---------------------------------------- 0.9/159.7 MB 9.5 MB/s eta 0:00:17
   ---------------------------------------- 1.5/159.7 MB 10.9 MB/s eta 0:00:15
    --------------------------------------- 2.2/159.7 MB 11.5 MB/s eta 0:00:14
    --------------------------------------- 2.9/159.7 MB 12.1 MB/s eta 0:00:13
    --------------------------------------- 3.5/159.7 MB 12.3 MB/s eta 0:00:13
   - -----

  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git 'C:\Users\14158\AppData\Local\Temp\pip-req-build-qv99aivg'


In [2]:
import os
import clip
import torch
from PIL import Image
from torchvision.datasets import ImageFolder
from torchvision import transforms
from torch.utils.data import DataLoader


c:\Users\14158\miniconda3\envs\sky\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# Path to your dataset
dataset_path = 'C:/Users/14158/BrodyCode/HackDavis/Datasets/GarbageDatasets'

# Transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize the images to 224x224 pixels
    transforms.ToTensor(),          # Convert images to PyTorch tensors
    transforms.Normalize(mean=[0.48145466, 0.4578275, 0.40821073],  # CLIP's mean
                         std=[0.26862954, 0.26130258, 0.27577711])  # CLIP's std
])

# Load the dataset
dataset = ImageFolder(root=dataset_path, transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)


In [5]:
# Load the model
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load('ViT-B/32', device)


100%|███████████████████████████████████████| 338M/338M [00:27<00:00, 13.0MiB/s]


In [15]:
def predict(image_path, text_labels):
    # Load the image and preprocess it
    image = preprocess(Image.open(image_path)).unsqueeze(0).to(device)
    
    # Prepare text prompts for classification and tokenize
    text_inputs = clip.tokenize(text_labels).to(device)
    
    # Forward pass through CLIP model with both image and text inputs
    with torch.no_grad():
        # It is important to pass both image and text to the model
        logits_per_image, logits_per_text = model(image, text_inputs)
        # Calculate the softmax to get probabilities
        probs = logits_per_image.softmax(dim=-1).cpu().numpy()
    
    # Determine the highest probability index and select the corresponding label
    top_prediction = text_labels[probs.argmax()]

    return top_prediction

In [37]:
# Example of classifying an image
"TEST SAMPLE OF ONE IMAGE"
image_path = 'C:/Users/14158/BrodyCode/HackDavis/Datasets/TestDataset/battery_15.jpg'

text_labels = ["battery", "biological", "cardboard", "clothes", "glass", "metal", "paper", "plastic", "shoes", "trash"]


predicted_class = predict(image_path,text_labels)
print(type(predicted_class))
print(f"Predicted class of image is : {predicted_class}")

<class 'str'>
Predicted class of image is : metal


In [60]:
"Cell that creates TestDataset Folder: "
import os
import shutil
import random

def randomly_select_images(source_dir, target_dir, num_images=200):
    # Collect all image paths from each subfolder in the main directory
    all_images = []
    # List subdirectories in the main directory
    subdirs = [os.path.join(source_dir, d) for d in os.listdir(source_dir) if os.path.isdir(os.path.join(source_dir, d))]
    
    # Loop through each subdirectory to gather image files
    for subdir in subdirs:
        for file in os.listdir(subdir):
            if file.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
                all_images.append(os.path.join(subdir, file))

    # Ensure there are enough images to select from
    if len(all_images) < num_images:
        raise ValueError(f"Not enough images available to select {num_images}; only {len(all_images)} available.")

    # Randomly select a total of 200 images across all subfolders
    selected_images = random.sample(all_images, num_images)

    # Ensure the target directory exists
    os.makedirs(target_dir, exist_ok=True)

    # Copy selected images to the target directory
    for image in selected_images:
        shutil.copy(image, target_dir)

    print(f"Successfully copied {len(selected_images)} images to {target_dir}")



source_directory = 'C:/Users/14158/BrodyCode/HackDavis/Datasets/GarbageDatasets'  # Path to the directory containing folders of images
target_directory = 'C:/Users/14158/BrodyCode/HackDavis/Datasets/TestDataset'       # Path where the selected images will be stored

# Call the function
randomly_select_images(source_directory, target_directory)



Successfully copied 200 images to C:/Users/14158/BrodyCode/HackDavis/Datasets/TestDataset


In [59]:
import os

def rename_files(directory):
    # Iterate over all files in the directory
    for filename in os.listdir(directory):
        # Check if 'trash' is in the filename
        if 'battery' in filename:
            # Create the new filename by replacing 'trash' with 'household_waste'
            new_filename = filename.replace('battery', 'e_waste')
            # Construct full file paths
            old_file_path = os.path.join(directory, filename)
            new_file_path = os.path.join(directory, new_filename)
            # Rename the file
            os.rename(old_file_path, new_file_path)
            print(f"Renamed '{filename}' to '{new_filename}'")

# Directory containing the files to be renamed
directory_path = 'C:/Users/14158/BrodyCode/HackDavis/Datasets/GarbageDatasets/battery'

# Call the function
rename_files(directory_path)


Renamed 'battery_1.jpg' to 'e_waste_1.jpg'
Renamed 'battery_10.jpg' to 'e_waste_10.jpg'
Renamed 'battery_100.jpg' to 'e_waste_100.jpg'
Renamed 'battery_101.jpg' to 'e_waste_101.jpg'
Renamed 'battery_102.jpg' to 'e_waste_102.jpg'
Renamed 'battery_103.jpg' to 'e_waste_103.jpg'
Renamed 'battery_104.jpg' to 'e_waste_104.jpg'
Renamed 'battery_105.jpg' to 'e_waste_105.jpg'
Renamed 'battery_106.jpg' to 'e_waste_106.jpg'
Renamed 'battery_107.jpg' to 'e_waste_107.jpg'
Renamed 'battery_108.jpg' to 'e_waste_108.jpg'
Renamed 'battery_109.jpg' to 'e_waste_109.jpg'
Renamed 'battery_11.jpg' to 'e_waste_11.jpg'
Renamed 'battery_110.jpg' to 'e_waste_110.jpg'
Renamed 'battery_111.jpg' to 'e_waste_111.jpg'
Renamed 'battery_112.jpg' to 'e_waste_112.jpg'
Renamed 'battery_113.jpg' to 'e_waste_113.jpg'
Renamed 'battery_114.jpg' to 'e_waste_114.jpg'
Renamed 'battery_115.jpg' to 'e_waste_115.jpg'
Renamed 'battery_116.jpg' to 'e_waste_116.jpg'
Renamed 'battery_117.jpg' to 'e_waste_117.jpg'
Renamed 'battery_118.

In [62]:
def test_model_accuracy(directory, text_labels):
    # Initialize counters for each category
    category_counts = {label: {'success': 0, 'total': 0} for label in text_labels}
    total_count = 0
    success_count = 0

    # List all image files in the directory
    files = [f for f in os.listdir(directory) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif'))]

    # Iterate over each file
    for file in files:
        image_path = os.path.join(directory, file)
        predicted_class = predict(image_path, text_labels)  # Assuming predict function is defined and available
        
        # Determine if a prediction is successful
        if predicted_class in file.lower():
            success_count += 1
            print(f"File '{file}': Success (Predicted: {predicted_class})")
        else:
            print(f"File '{file}': Failure (Predicted: {predicted_class})")

        # Increment total count for each category present in the filename
        for label in text_labels:
            if label in file.lower():
                category_counts[label]['total'] += 1
                if predicted_class == label:
                    category_counts[label]['success'] += 1

        total_count += 1

    # Calculate and print the accuracy for each category
    for label, counts in category_counts.items():
        if counts['total'] > 0:
            accuracy = (counts['success'] / counts['total']) * 100
            print(f"Accuracy for {label}: {accuracy:.2f}% ({counts['success']}/{counts['total']})")
        else:
            print(f"Accuracy for {label}: N/A (no images found)")

    # Calculate and print the total accuracy
    total_accuracy = (success_count / total_count) * 100 if total_count > 0 else 0
    print(f"Total Accuracy: {total_accuracy:.2f}% ({success_count}/{total_count})")

# Directory containing the test images
source_directory = 'C:/Users/14158/BrodyCode/HackDavis/Datasets/TestDataset'

# List of possible labels
text_labels = ["e_waste", "shoes", "clothes","biological", "glass", "metal", "paper", "plastic",  "hygienic"]

# Call the testing function
test_model_accuracy(source_directory, text_labels)

File 'biological_191.jpg': Failure (Predicted: plastic)
File 'biological_267.jpg': Success (Predicted: biological)
File 'biological_286.jpg': Success (Predicted: biological)
File 'biological_365.jpg': Success (Predicted: biological)
File 'biological_489.jpg': Failure (Predicted: plastic)
File 'biological_590.jpg': Success (Predicted: biological)
File 'biological_652.jpg': Success (Predicted: biological)
File 'biological_669.jpg': Failure (Predicted: plastic)
File 'biological_861.jpg': Success (Predicted: biological)
File 'biological_904.jpg': Failure (Predicted: metal)
File 'clothes_1257.jpg': Success (Predicted: clothes)
File 'clothes_1421.jpg': Success (Predicted: clothes)
File 'clothes_1541.jpg': Success (Predicted: clothes)
File 'clothes_2060.jpg': Success (Predicted: clothes)
File 'clothes_2064.jpg': Success (Predicted: clothes)
File 'clothes_219.jpg': Success (Predicted: clothes)
File 'clothes_2224.jpg': Success (Predicted: clothes)
File 'clothes_2300.jpg': Success (Predicted: cl